# Question-Answering on Private Documents

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [4]:
# !pip install pypdf -q
# !pip install docx2txt -q
# !pip install wikipedia -q

### Loading Documents

In [6]:
# loading PDF, DOCX and TXT files as LangChain Documents
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data
  

In [7]:
# wikipedia
def load_from_wikipedia(query, lang='en', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data


### Chunking Data

In [8]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks
    

### Calculating Cost

In [9]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}')

### Embedding and Uploading to a Vector Database (Pinecone)

In [10]:
def insert_or_fetch_embeddings(index_name):
    import pinecone
    from langchain.vectorstores import Pinecone
    from langchain.embeddings.openai import OpenAIEmbeddings
    
    embeddings = OpenAIEmbeddings()
    
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))
    
    if index_name in pinecone.list_indexes():
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        print(f'Creating index {index_name} and embeddings ...', end='')
        pinecone.create_index(index_name, dimension=1536, metric='cosine')
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')
        
    return vector_store
    

In [11]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))
    
    if index_name == 'all':
        indexes = pinecone.list_indexes()
        print('Deleting all indexes ... ')
        for index in indexes:
            pinecone.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pinecone.delete_index(index_name)
        print('Ok')
    

### Asking and Getting Answers

In [12]:
def ask_and_get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    
    answer = chain.run(q)
    return answer
    
    
def ask_with_memory(vector_store, question, chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import ChatOpenAI
    
    llm = ChatOpenAI(temperature=1)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    result = crc({'question': question, 'chat_history': chat_history})
    chat_history.append((question, result['answer']))
    
    return result, chat_history
    

### Running Code

In [49]:
data = load_document('files/us_constitution.pdf')
# print(data[1].page_content)
# print(data[10].metadata)

print(f'You have {len(data)} pages in your data')
print(f'There are {len(data[20].page_content)} characters in the page')

Loading files/us_constitution.pdf
You have 41 pages in your data
There are 1137 characters in the page


In [39]:
## Different document formats

# data = load_document('files/the_great_gatsby.docx')
# print(data[0].page_content)

# data = load_from_wikipedia('GPT-4', 'de')
# print(data[0].page_content)

In [50]:
chunks = chunk_data(data)
print(len(chunks))
# print(chunks[10].page_content)

190


In [51]:
print_embedding_cost(chunks)

Total Tokens: 16711
Embedding Cost in USD: 0.006684


In [52]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


In [53]:
index_name = 'askadocument'
vector_store = insert_or_fetch_embeddings(index_name)

Creating index askadocument and embeddings ...Ok


In [56]:
q = 'What is the whole document about?'
answer = ask_and_get_answer(vector_store, q)
print(answer)

The document is about the United States Constitution. It states the purpose of the Constitution, which is to create a more perfect union, establish justice, ensure domestic tranquility, provide for the common defense, promote the general welfare, and secure the blessings of liberty. It also establishes the authority of the Constitution, stating that it is the supreme law of the land and that all treaties made under the authority of the United States are also binding. Additionally, it states that citizens of each state are entitled to all privileges and immunities of citizens in other states, and that laws regarding how acts, records, and proceedings are proved and their effect are to be determined by general laws.


In [57]:
import time
i = 1
print('Write Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break
    
    answer = ask_and_get_answer(vector_store, q)
    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')

    

Write Quit or Exit to quit.


Question #1:  how many articles are there?



Answer: There are three articles mentioned in the provided context.

 -------------------------------------------------- 



Question #2:  articles in the us constitution?



Answer: There are many articles in the US Constitution. Here are a few examples:

- Article I: Establishes the legislative branch of the federal government, known as Congress.
- Article II: Establishes the executive branch of the federal government, headed by the President.
- Article III: Establishes the judicial branch of the federal government, including the Supreme Court.
- Article IV: Addresses the relationship between the states and the federal government.
- Article V: Outlines the process for amending the Constitution.
- Article VI: Establishes that the Constitution is the supreme law of the land.
- Article VII: Explains the process for ratifying the Constitution.

There are a total of seven articles in the US Constitution.

 -------------------------------------------------- 



Question #3:  quit


Quitting ... bye bye!


In [37]:
## Wikipedia Example
# delete_pinecone_index()

# data = load_from_wikipedia('GPT-4', 'en')
# chunks = chunk_data(data)
# index_name = 'chatgpt'
# vector_store = insert_or_fetch_embeddings(index_name)

# q = 'What is gpt-4?'
# answer = ask_and_get_answer(vector_store, q)
# print(answer)

In [58]:
# asking with memory
chat_history = []
question = 'How many amendments are in the U.S. Constitution?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
print(chat_history)

There are currently 27 amendments in the U.S. Constitution.
[('How many amendments are in the U.S. Constitution?', 'There are currently 27 amendments in the U.S. Constitution.')]


In [60]:
question = 'Multiply that number by 2'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
print(chat_history)


There are currently 27 amendments in the U.S. Constitution. Multiplying this by 2 would give us a result of 54.
[('How many amendments are in the U.S. Constitution?', 'There are currently 27 amendments in the U.S. Constitution.'), ('Multiply that number by 2', 'The given context does not mention any specific number or quantity to multiply by 2. Therefore, it is not possible to determine the result of multiplying that number by 2.'), ('Multiply that number by 2', 'There are currently 27 amendments in the U.S. Constitution. Multiplying this by 2 would give us a result of 54.')]


### Ask with Memory Loop

In [ ]:
import time
i = 1

chat_history = []

print("Write Quit or Exit to quit")
while True:
    q = input(f"Question #{i}")
    i = i + 1
    if q.lower() in ["quit","exit"]:
        print("Qutting")
        time.sleep(2)
        break
    result, _ = ask_with_memory(vector_store, q, chat_history)
    print (result['answer'])
    print("----------------------------------------------------------------------")